<a href="https://colab.research.google.com/github/JesseTNRoberts/AAAI-paper-2023/blob/main/pop_analysis_basic_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install nbformat --upgrade
!pip install seaborn
!pip install researchpy


#TODO: Jesse pointed out that the first cell in one of the analysis notebooks from last time generated whisker plots that might be useful for eyeballing the presence of the effect before running Wilcoxon. Look into it

^C
                                              0.0/77.6 kB ? eta -:--:--
     ---------------------------------------- 77.6/77.6 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.9.1
    Uninstalling nbformat-5.9.1:
      Successfully uninstalled nbformat-5.9.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.10 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Assuming your CSV file is named "data.csv"
import pandas as pd
from tqdm import tqdm

import pandas as pd
import plotly.graph_objects as go
from  plotly.subplots import make_subplots
import plotly.colors as pc
import researchpy as rp
import numpy as np
import random
import ast

from scipy.stats import ttest_ind, ttest_ind_from_stats, combine_pvalues
from scipy.special import stdtr
from scipy.stats import wilcoxon, ranksums
from scipy.stats import mannwhitneyu
import scipy

loc = "../Experiments/data/results"

In [3]:
models = ['bert-base-uncased', 'bert-large-uncased', 'distilbert-base-uncased', 'distilgpt2',
          'gpt2-medium', 'gpt2', 'openai-gpt', 'roberta-base', 'roberta-large']

def load_data(model_name, sample_num = 500, sent_struct = 'comb', reduction = 'mean'):
    da_datasets = {
        1500: 'CORE_dative_1500sampled_popLM',
        15000: 'CORE_dative_15000sampled_10-1_popLM'
    }
    tr_datasets = {
        1500: 'CORE_transitive_1500sampled_popLM',
        15000: 'CORE_transitive_15000sampled_10-1_popLM'
    }

    res_cols = ['tx', 'ty', 'px-tx', 'px-ty', 'py-tx', 'py-ty']

    if sent_struct in ['da', 'comb']:
        da_data_raw = pd.read_csv(loc+'/'+da_datasets[sample_num]+'/'+model_name+'.csv')
        da_data = {'ds_name': da_data_raw['ds_name']}
    if sent_struct in ['tr', 'comb']:
        tr_data_raw = pd.read_csv(loc+'/'+tr_datasets[sample_num]+'/'+model_name+'.csv')
        tr_data = {'ds_name': tr_data_raw['ds_name']}

    if reduction == 'mean':
        select_reduction = 1
    elif reduction == 'sum':
        select_reduction = 0

    for rc in res_cols:
        if sent_struct in ['da', 'comb']:
            da_data[rc] = [[t[select_reduction] for t in m] for m in da_data_raw[rc + ' results'].apply(ast.literal_eval)]
        if sent_struct in ['tr', 'comb']:
            tr_data[rc] = [[t[select_reduction] for t in m] for m in tr_data_raw[rc + ' results'].apply(ast.literal_eval)]

    if sent_struct == 'da':
        return pd.DataFrame(da_data)
    elif sent_struct == 'tr':
        return pd.DataFrame(tr_data)
    elif sent_struct == 'comb':
        return pd.concat([pd.DataFrame(da_data), pd.DataFrame(tr_data)])

def cohend(d1, d2) -> float:
  # calculate the size of samples
  n1, n2 = len(d1), len(d2)

  # calculate the variance of the samples
  s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)

  # calculate the pooled standard deviation
  s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))

  # calculate the means of the samples
  u1, u2 = np.mean(d1), np.mean(d2)

  # return the effect size
  return (u1 - u2) / s

def add_exp_num(df):
  df['experiment_number'] = range(len(df))


def add_stats(df):
  add_exp_num(df)

  df['tx-mean'] = df.apply(
      lambda row: np.mean(row['tx']),
      axis=1)
  df['ty-mean'] = df.apply(
      lambda row: np.mean(row['ty']),
      axis=1)
  df['tx-std'] = df.apply(
      lambda row: np.std(row['tx']),
      axis=1)
  df['ty-std'] = df.apply(
      lambda row: np.std(row['ty']),
      axis=1)

  df['cohend - ntx-px-tx'] = df.apply(
      lambda row: cohend(row['tx'], row['px-tx']),
      axis=1)
  df['cohend - ntx-py-tx'] = df.apply(
      lambda row: cohend(row['tx'], row['py-tx']),
      axis=1)
  df['cohend - nty-py-ty'] = df.apply(
      lambda row: cohend(row['ty'], row['py-ty']),
      axis=1)
  df['cohend - nty-px-ty'] = df.apply(
      lambda row: cohend(row['ty'], row['px-ty']),
      axis=1)
  df['cohend - px-py-tx'] = df.apply(
      lambda row: cohend(row['px-tx'], row['py-tx']),
      axis=1)
  df['cohend - py-py-ty'] = df.apply(
      lambda row: cohend(row['py-ty'], row['px-ty']),
      axis=1)
  #TODO should calculate for set 'tx'+'ty' vs 'px-tx'+'py-ty'
#   df['cohend - np-same'] = df.apply(
#       lambda row: cohend(row['ty'], row['py-ty']),
#       axis=1)
#   df['cohend - np-diff'] = df.apply(
#       lambda row: cohend(row['ty'], row['py-ty']),
#       axis=1)

  df['change - npx-px-tx'] = df.apply(
      lambda row: np.subtract(row['tx'], row['px-tx']),
      axis=1)
  df['change - npx-py-tx'] = df.apply(
      lambda row: np.subtract(row['tx'], row['py-tx']),
      axis=1)
  df['change - npy-py-ty'] = df.apply(
      lambda row: np.subtract(row['ty'], row['py-ty']),
      axis=1)
  df['change - npy-px-ty'] = df.apply(
      lambda row: np.subtract(row['ty'], row['px-ty']),
      axis=1)
  df['change - px-tx-ty'] = df.apply(
      lambda row: np.subtract(row['px-tx'], row['px-ty']),
      axis=1)
  df['change - py-ty-tx'] = df.apply(
      lambda row: np.subtract(row['py-ty'], row['py-tx']),
      axis=1)
  
def run_pipeline(model_name, **kwargs):
    df = load_data(model_name, **kwargs)
    print(df.iloc([0]))
    add_stats(df)
    return df

In [4]:
temp_data = load_data('bert-base-uncased', 1500, 'comb')

In [5]:
add_stats(temp_data)
temp_data[temp_data['experiment_number'].isin([0,50])]

,ds_name,tx,ty,px-tx,px-ty,py-tx,py-ty,experiment_number,tx-mean,ty-mean,...,cohend - nty-py-ty,cohend - nty-px-ty,cohend - px-py-tx,cohend - py-py-ty,change - npx-px-tx,change - npx-py-tx,change - npy-py-ty,change - npy-px-ty,change - px-tx-ty,change - py-ty-tx
0,core-da,"[-3.3338139851888022, -3.1904694769117565, -2....","[-5.069084048271179, -4.770589590072632, -4.94...","[-2.7957226435343423, -2.8790257771809897, -2....","[-4.818917334079742, -4.1944732666015625, -4.0...","[-2.859648757510715, -2.378713289896647, -2.79...","[-4.366383135318756, -4.394528687000275, -4.53...",0,-3.130551,-4.855303,...,-1.500762,-1.119795,-0.222938,0.396177,"[-0.5380913416544599, -0.3114436997307668, -0....","[-0.4741652276780872, -0.8117561870151095, -0....","[-0.7027009129524231, -0.3760609030723572, -0....","[-0.25016671419143677, -0.5761163234710693, -0...","[2.0231946905454, 1.3154474894205728, 1.572688...","[-1.506734377808041, -2.0158153971036277, -1.7..."
50,core-da,"[-3.6117113961113825, -3.652260144551595, -3.5...","[-3.1355345845222473, -3.3819990158081055, -3....","[-4.16769536336263, -3.8866937425401478, -3.96...","[-3.603093683719635, -3.3670091032981873, -3.8...","[-3.9463238186306424, -4.192515002356635, -4.0...","[-3.53827565908432, -3.973977744579315, -3.826...",50,-3.618875,-3.327953,...,2.652311,1.979082,0.200034,-0.399432,"[0.5559839672512474, 0.23443359798855257, 0.37...","[0.33461242251925993, 0.5402548578050399, 0.43...","[0.40274107456207275, 0.5919787287712097, 0.47...","[0.4675590991973877, -0.014989912509918213, 0....","[-0.5646016796429949, -0.5196846392419605, -0....","[0.40804815954632234, 0.21853725777731992, 0.1..."


In [3]:
def sweep_pops_ttest_ctrl_vs_eff(model_name, raw_data, col_groups, pop_step = 1):
    '''
        col groups should be a list of 2-tuples where each tuple has the form (control_col_name, effect_col_name)
    '''
    if 'experiment_number' not in raw_data:
        add_exp_num(raw_data)

    fig = make_subplots(rows = 1, cols = 2)

    print(f'Evaluating {model_name} results')
    for i, (control_col, effect_col) in enumerate(col_groups):
        print(f'\tEvaluating columns: {control_col} vs. {effect_col}')
        data = raw_data.explode([control_col, effect_col])
        data['member id'] = data.groupby(['experiment_number']).cumcount()+1
        data[control_col] = data[control_col].astype('float')
        data[effect_col] = data[effect_col].astype('float')

        num_experiments = max(data['experiment_number']) + 1
        
        avg_wtts = []
        p_vals = []
        for pop_size in tqdm(range(2, max(data['member id']) + 1, pop_step), desc = 'population sizes', leave=False):
            out = []
            for exp_num in range(num_experiments):
                data_exp = data[data['experiment_number'] == exp_num]
                out.append(ttest_ind(
                    data_exp[data_exp['member id'] <= pop_size][control_col],
                    data_exp[data_exp['member id'] <= pop_size][effect_col],
                    equal_var = False
                ))
            
            out_stats = [k.statistic for k in out]
            avg_wtts.append(sum(out_stats) / len(out_stats))
            p_vals.append(scipy.stats.gmean([k.pvalue for k in out]) * pop_size)
        
        color = pc.qualitative.Plotly[i]
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = avg_wtts,
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}'),
                row = 1, col = 1
        )
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = p_vals,
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}',
                showlegend = False),
                row = 1, col = 2    
        )
    fig.update_layout(
        title_text=f"T-test results for all experiments, by pop. size<br />model: {model_name}"
    )
    fig.update_xaxes(title_text="Pop. size", row = 1, col = 1)
    fig.update_yaxes(title_text="T-test values (avg. over experiments)", row = 1, col = 1)

    fig.update_xaxes(title_text="Pop. size", row = 1, col = 2)
    fig.update_yaxes(title_text="T-test p-values (avg. over experiments)", row = 1, col = 2)
    
    fig.show()

In [4]:
models = ["openai-gpt",
          "gpt2",
          "gpt2-medium",
          "distilgpt2",
          "bert-base-uncased",
          "bert-large-uncased",
          "roberta-base",
          "roberta-large",
          "distilbert-base-uncased"]
column_groups = [(   'tx', 'px-tx'),
                 (   'tx', 'py-tx'),
                 ('py-tx', 'px-tx'),
                 (   'ty', 'py-ty'),
                 (   'ty', 'px-ty'),
                 ('px-ty', 'py-ty')]
pop_step = 1
dataset_size_variant = 1500 # alternatively 15000 if data is included
dataset_struct_variant = 'comb' # alternatively 'da' or 'tr'

for model_name in models:
    temp_data = load_data(model_name, dataset_size_variant, dataset_struct_variant)
    sweep_pops_ttest_ctrl_vs_eff(model_name, temp_data, column_groups, pop_step)

Evaluating openai-gpt results
	Evaluating columns: tx vs. px-tx


	Evaluating columns: tx vs. py-tx


	Evaluating columns: py-tx vs. px-tx


	Evaluating columns: ty vs. py-ty


	Evaluating columns: ty vs. px-ty


...

	Evaluating columns: tx vs. py-tx


	Evaluating columns: py-tx vs. px-tx


	Evaluating columns: ty vs. py-ty


	Evaluating columns: ty vs. px-ty


	Evaluating columns: px-ty vs. py-ty


In [61]:
def sweep_pops_ttest_ctrl_vs_eff_diff_mean(model_name, raw_data, col_groups, pop_step = 1):
    '''
        col groups should be a list of 2-tuples where each tuple has the form (control_col_name, effect_col_name)
    '''
    if 'experiment_number' not in raw_data:
        add_exp_num(raw_data)

    fig = make_subplots(rows = 2, cols = 3)

    print(f'Evaluating {model_name} results')
    for i, (control_col, effect_col) in enumerate(col_groups):
        print(f'\tEvaluating columns: {control_col} vs. {effect_col}')
        data = raw_data.explode([control_col, effect_col])
        data['member id'] = data.groupby(['experiment_number']).cumcount()+1
        data[control_col] = data[control_col].astype('float')
        data[effect_col] = data[effect_col].astype('float')

        num_experiments = max(data['experiment_number']) + 1
        
        p_vals = []

        for pop_size in tqdm(range(2, max(data['member id']) + 1, pop_step), desc = 'population sizes', leave=False):
            data_mem = data[data['member id'] <= pop_size]
            p_vals.append([ttest_ind(data_mem[data_mem['experiment_number'] == exp_num][control_col],
                                     data_mem[data_mem['experiment_number'] == exp_num][effect_col],
                                     equal_var = False).pvalue for exp_num in range(num_experiments)])
        
        color = pc.qualitative.Plotly[i]
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = [sum(p) / len(p) for p in p_vals],
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}',
                showlegend = False),
                row = 1, col = 1
        )
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = [scipy.stats.gmean(p) for p in p_vals],
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}',
                showlegend = False),
                row = 1, col = 2
        )
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = [scipy.stats.hmean(p) for p in p_vals],
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}'),
                row = 1, col = 3
        )
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = [scipy.stats.combine_pvalues(p, method='fisher')[1] for p in p_vals],
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}',
                showlegend = False),
                row = 2, col = 1
        )
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = [scipy.stats.combine_pvalues(p, method='pearson')[1] for p in p_vals],
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}',
                showlegend = False),
                row = 2, col = 2
        )
        fig.add_trace(
            go.Scatter(
                x = np.arange(2, max(data['member id']) + 1, pop_step),
                y = [scipy.stats.combine_pvalues(p, method='mudholkar_george')[1] for p in p_vals],
                name=f'{control_col} vs. {effect_col}',
                line_color = color,
                legendgroup = f'{control_col}{effect_col}',
                showlegend = False),
                row = 2, col = 3
        )


    fig.update_layout(
        title_text=f"T-test p-value results for all experiments, by pop. size<br />model: {model_name}"
    )
    fig.update_xaxes(title_text="Pop. size", row = 1, col = 1)
    fig.update_yaxes(title_text="arithemetic mean", row = 1, col = 1)

    fig.update_xaxes(title_text="Pop. size", row = 1, col = 2)
    fig.update_yaxes(title_text="geometric mean", row = 1, col = 2)

    fig.update_xaxes(title_text="Pop. size", row = 1, col = 3)
    fig.update_yaxes(title_text="harmonic mean", row = 1, col = 3)

    fig.update_xaxes(title_text="Pop. size", row = 2, col = 1)
    fig.update_yaxes(title_text="fisher's method", row = 2, col = 1)

    fig.update_xaxes(title_text="Pop. size", row = 2, col = 2)
    fig.update_yaxes(title_text="pearson's method", row = 2, col = 2)

    fig.update_xaxes(title_text="Pop. size", row = 2, col = 3)
    fig.update_yaxes(title_text="mudholkar-george's method", row = 2, col = 3)
    
    fig.show()

In [35]:
bert_df = load_data('bert-base-uncased', dataset_size_variant, dataset_struct_variant)

In [36]:
gpt2_df = load_data('gpt2', dataset_size_variant, dataset_struct_variant)

In [64]:
column_groups = [('tx', 'px-tx'), ('tx', 'py-tx'), ('py-tx', 'px-tx')]
pop_step = 1
dataset_size_variant = 1500 # alternatively 15000 if data is included
dataset_struct_variant = 'comb' # alternatively 'da' or 'tr'

sweep_pops_ttest_ctrl_vs_eff_diff_mean('bert-base-uncased', bert_df, column_groups, pop_step)
# sweep_pops_ttest_ctrl_vs_eff_diff_mean('gpt2', gpt2_df, column_groups, pop_step)


Evaluating bert-base-uncased results
	Evaluating columns: tx vs. px-tx


	Evaluating columns: tx vs. py-tx


	Evaluating columns: py-tx vs. px-tx


In [28]:
def get_chi_squared(data, control_col, effect_col, threshold = 0.05, num_subjects = -1):
    data = data.explode([control_col, effect_col])
    data['member id'] = data.groupby(['experiment_number']).cumcount()+1
    data[control_col] = data[control_col].astype('float')
    data[effect_col] = data[effect_col].astype('float')

    num_experiments = max(data['experiment_number']) + 1

    if num_subjects == -1:
        num_subjects = max(data['member id']) - 1
    data_subs = data[data['member id'].isin(random.sample(range(max(data['member id'])), num_subjects))]

    # for exp_num in range(num_experiments):
    #             data_exp = data[data['experiment_number'] == exp_num]
    #             out.append(ttest_ind(
    #                 data_exp[data_exp['member id'] <= pop_size][control_col],
    #                 data_exp[data_exp['member id'] <= pop_size][effect_col],
    #                 equal_var = False
    #             ))

    out = [ttest_ind(data_subs[data_subs['experiment_number'] == exp_num][control_col],
                     data_subs[data_subs['experiment_number'] == exp_num][effect_col]) for exp_num in range(num_experiments)]
    out_stats = [i.statistic for i in out]
    out_p = [i.pvalue for i in out]

    print(f'columns: {control_col} vs {effect_col}')

    return sum(out_stats) / len(out_stats), -2 * sum(map(np.log, out_p)), sum(out_p) / len(out_p), scipy.stats.hmean(out_p), scipy.stats.gmean(out_p)

In [6]:
temp_data = load_data('bert-base-uncased', 1500, 'comb')
add_exp_num(temp_data)

In [30]:
print(get_chi_squared(temp_data,    'tx', 'px-tx', num_subjects = 5))
print(get_chi_squared(temp_data,    'tx', 'py-tx', num_subjects = 5))
print(get_chi_squared(temp_data, 'py-tx', 'px-tx', num_subjects = 5))

columns: tx vs px-tx
(1.110985607928546, 16658.408474338066, 0.2607519746873797, 0.0002500597505303454, 0.06226216055779539)
columns: tx vs py-tx
(0.8327987731029449, 19688.66187438225, 0.23748728724744705, 2.425200274020957e-05, 0.03757397412680261)
columns: py-tx vs px-tx
(0.4392711224048231, 10092.698399767824, 0.3824358511564181, 0.005197329174959288, 0.18597995126781267)


In [11]:
def placebo(data, control_col, treatment_col, placebo_col, comp_fn = pd.DataFrame.__lt__, threshold = 0.05):
    data = data.explode([placebo_col, treatment_col, control_col])
    data['member id'] = data.groupby(['experiment_number']).cumcount()+1
    data[placebo_col] = data[placebo_col].astype('float')
    data[treatment_col] = data[treatment_col].astype('float')
    data[control_col] = data[control_col].astype('float')

    data['treatment non-log diff'] = data.apply(
        lambda x: 2 ** x[treatment_col] - 2 ** x[control_col],
        axis = 1
    )
    data['placebo non-log diff'] = data.apply(
        lambda x: 2 ** x[placebo_col] - 2 ** x[control_col],
        axis = 1
    )

    data['treatment diff'] = data[treatment_col] - data[control_col]
    data['placebo diff'] = data[placebo_col] - data[control_col]


    data['naive'] = comp_fn(data['treatment non-log diff'], data['placebo non-log diff'])
    print('classifier accuracy', data[data['naive'] == 1]['naive'].count() / data['naive'].count())
    print('treatment mean when correct', data[data['naive'] == 1]['treatment non-log diff'].mean())
    print('placebo mean when correct', data[data['naive'] == 1]['placebo non-log diff'].mean())
    print('treatment mean when incorrect', data[data['naive'] == 0]['treatment non-log diff'].mean())
    print('placebo mean when incorrect', data[data['naive'] == 0]['placebo non-log diff'].mean())


    print((data.groupby('experiment_number')['naive'].sum() > 25).mean(), data.groupby('experiment_number')['naive'].first().mean())

    num_experiments = max(data['experiment_number']) + 1

    out = [ranksums(data[data['experiment_number'] == exp_num]['treatment diff'],
                    data[data['experiment_number'] == exp_num]['placebo diff'],
                    alternative = 'two-sided') for exp_num in range(num_experiments)]
    # out = [wilcoxon(data[data['experiment_number'] == exp_num]['treatment diff'],
    #                  data[data['experiment_number'] == exp_num]['placebo diff']) for exp_num in range(num_experiments)]
    
    print(out[:5])

    print('ttest (W): ', wilcoxon(data['treatment diff'], data['placebo diff']))
    print('ttest (M): ', mannwhitneyu(data['treatment diff'], data['placebo diff']))

    out_stats = [i.statistic for i in out]
    out_p = [i.pvalue for i in out]

    return sum(out_stats) / len(out_stats), -2 * sum(map(np.log, out_p)), scipy.stats.hmean(out_p), (scipy.stats.hmean(out_p) * num_experiments)

In [12]:
placebo(temp_data, 'tx', 'px-tx', 'py-tx')

classifier accuracy 0.5615333333333333
treatment mean when correct -0.010299226403344765
placebo mean when correct -0.0006420775262770597
treatment mean when incorrect -0.0006109261701562938
placebo mean when incorrect -0.009282936909162105
0.6136666666666667 0.5593333333333333
[RanksumsResult(statistic=-0.4618859316556266, pvalue=0.6441631231916007), RanksumsResult(statistic=-2.4059431365345323, pvalue=0.016130775289776193), RanksumsResult(statistic=1.9095881055016202, pvalue=0.0561862677962898), RanksumsResult(statistic=0.40673537265196963, pvalue=0.6842023576175498), RanksumsResult(statistic=1.5717909316042218, pvalue=0.11599904533016406)]
ttest (W):  WilcoxonResult(statistic=4643208207.0, pvalue=0.0)
ttest (M):  MannwhitneyuResult(statistic=10545316008.5, pvalue=5.368143252308639e-194)


(-0.7930926137520888,
 25177.332587077282,
 2.2877627841918018e-14,
 6.863288352575405e-11)

In [344]:
temp_data = load_data('distilbert-base-uncased', 1500, 'comb')
add_stats(temp_data)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [376]:
placebo(temp_data, 'tx', 'px-tx', 'py-tx', pd.DataFrame.__gt__, 0.05) 

classifier accuracy 0.5345266666666667
treatment mean when correct 0.004141892462799024
placebo mean when correct -0.0027474765381995045
treatment mean when incorrect -0.00047682679236933157
placebo mean when incorrect 0.005808027905313303
0.5016666666666667 0.5353333333333333
[RanksumsResult(statistic=-0.2550713353919132, pvalue=0.7986680030925278), RanksumsResult(statistic=-0.19302695651279916, pvalue=0.8469378568304889), RanksumsResult(statistic=1.8889066458752488, pvalue=0.05890433880651089), RanksumsResult(statistic=-0.5377179502856548, pvalue=0.5907717847981475), RanksumsResult(statistic=0.12408875775822803, pvalue=0.9012449993700387)]
ttest (W):  WilcoxonResult(statistic=4951226230.5, pvalue=0.0)
ttest (M):  MannwhitneyuResult(statistic=11658801686.0, pvalue=1.411832492973884e-66)
columns: tx vs px-tx
stats for highest p-value
stat: 0.0	p-value: 1.0
stats for lowest p-value
stat: 8.520761366064992	p-value: 1.5850782976430648e-17


1.5850782976430648e-17 1.0 1820


(0.4750692131974603,
 25539.75479733534,
 3.433677290109917e-14,
 1.0301031870329751e-10)

In [366]:
placebo(temp_data, 'ty', 'py-ty', 'px-ty', pd.DataFrame.__gt__, 0.05)

classifier accuracy 0.52594
treatment mean when correct 0.007047191423697537
placebo mean when correct 0.0002597726574736503
treatment mean when incorrect -0.0019018348917625176
placebo mean when incorrect 0.006045238674982822
0.5866666666666667 0.5196666666666667
ttest (W):  WilcoxonResult(statistic=5185250272.5, pvalue=1.4260318226646994e-151)
ttest (M):  MannwhitneyuResult(statistic=11579646555.0, pvalue=6.414644070341553e-44)
columns: ty vs py-ty
stats for highest p-value
stat: 637.0	p-value: 0.9961489139450291
stats for lowest p-value
stat: 0.0	p-value: 7.556929455863566e-10


7.556929455863566e-10 0.9961489139450291 1407


(395.4336666666667,
 30252.032526789317,
 7.666178804256232e-08,
 0.00022998536412768697)

In [196]:
# temp_data = load_data('gpt2', 1500, 'da')
# add_stats(temp_data)
print(
    # get_chi_squared(temp_data,    'tx', 'px-tx', 0.1),
    # get_chi_squared(temp_data,    'tx', 'py-tx', 0.1),
    # get_chi_squared(temp_data,    'ty', 'py-ty', 0.1),
    # get_chi_squared(temp_data,    'ty', 'px-ty', 0.1),
    get_chi_squared(temp_data, 'px-tx', 'py-tx', 0.1),
    get_chi_squared(temp_data, 'py-ty', 'px-ty', 0.1), sep='\n')

1.0453686782659985e-45 0.9996629376616055 1084


7.277121572146858e-51 0.9995773554047027 852


None
None


In [181]:
out = pd.DataFrame()
for i, r in temp_data.iterrows():
    this = pd.DataFrame(list(zip([item for item in r['tx']], [item for item in r['px-tx']])), columns=['tx', 'pxtx'])
    this["id"] = range(1, this.shape[0] + 1)

    df2 = pd.melt(this, id_vars = "id", value_vars = ["tx", "pxtx"],
                   var_name = "priming", value_name = "prob")
    
    rp_out = rp.difference_test('prob ~ C(priming)', df2, equal_variances = True).conduct(effect_size=["Cohen's D"])#[1].set_index('Independent samples t-test with equal variances').T
    # out = pd.concat([out, rp_out.loc[:, ["Cohen's Ds", 'Two sided test p-value =', 'Difference < 0 p-value =', 'Difference > 0 p-value =']]])
    break
# rp_out.iloc[:,:]
rp_out

(       Name    N    Mean  Variance        SD      SE  95% Conf. Interval
 0      pxtx   50 -5.9837  0.019602  0.140006  0.0198  [-6.0235, -5.9439]
 1        tx   50 -7.7839  0.030516  0.174689  0.0247  [-7.8335, -7.7342]
 2  combined  100 -6.8838  0.843162  0.918239  0.0918   [-7.066, -6.7016]
 3      diff       1.8002                      0.0317     [1.7374, 1.863],
   Independent samples t-test with equal variances  Results
 0                          Difference (pxtx - tx)   1.8002
 1                            Degrees of freedom =  98.0000
 2                                             t =  56.8601
 3                        Two sided test p-value =   0.0000
 4                        Difference < 0 p-value =   1.0000
 5                        Difference > 0 p-value =   0.0000
 6                                      Cohen's Ds  11.3720)